In [1]:
import os
import random
import subprocess
import time

import numpy as np
from tqdm import tqdm, trange

In [2]:
def run_trial(configurations):
    theta, mapping, alpha, beta, lmbda = configurations
    header = "L,shift,alpha,beta,lambda,kappa\n"
    maple_executable_path = r"/home/conficker/programs/maple2023/bin/maple"
    # maple_file = "./volterra-population.mw"
    maple_file = "./kidder.mw"
    kappa = 0.5

    # Create the configuration file
    config_file_path = "config.txt"
    with open(config_file_path, "w") as file:
        rounded_values = [
            round(theta, 5),
            round(alpha, 5),
            round(beta, 5),
            round(lmbda, 5),
            int(mapping),
            kappa,
        ]
        formatted_values = ",".join(map(str, rounded_values))
        file.write(f"{header}{formatted_values}\n")

    # Run the Maple script
    process = subprocess.Popen(
        [maple_executable_path, maple_file],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )

    try:
        process.wait(timeout=2)  # Wait for the process to complete or timeout
        stdout, stderr = process.communicate()
        
        # Read the result from the output file
        result_file_path = "result.csv"
        try:    
            with open(result_file_path) as result_file:
                res = float(result_file.readline().strip())
            os.remove(result_file_path)  # Clean up the result file
        except (FileNotFoundError, ValueError):
            res = 1e100  # Failed to converge or file read error

        

    except subprocess.TimeoutExpired:
        res = 1e101  # Timeout error
        process.kill()  # Ensure the process is terminated
        stdout, stderr = process.communicate()

    # Log the results
    with open("log.txt", "a") as log_file:
        log_file.write(f"{configurations}, {res}, {time.time()}\n")

    os.remove(config_file_path)  # Clean up the config file

    return res

# Grid Search

In [3]:
import itertools

In [4]:
def grid_search(f, vardists):
    best_solution = None
    best_cost = float("inf")
    history = []

    # Generate all possible combinations of hyperparameters
    all_combinations = list(itertools.product(*vardists))
    t = tqdm(all_combinations, desc="Best cost", leave=True)

    for solution in t:
        cost = f(solution)
        history.append((solution, cost))

        if cost < best_cost:
            best_solution = solution
            best_cost = cost

        t.set_description(f"Best cost: {best_cost:.2e}")
        t.refresh()

    return best_solution, best_cost, history

In [5]:
vardists = (
    [2.5],  # theta:  [0.1, 10]
    [1, 2, 3],  # mapping
    [-0.5, 0],  # alpha:  [-1, 0]
    [-0.5, -0],  # beta:   [-1, 3]
    [0.5, 1],  # lambda: [0.1, 2]
)

best_solution, best_fitness, history = grid_search(run_trial, vardists)

Best cost: 3.00e-04: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:13<00:00,  1.80it/s]


# Random Search

In [6]:
from scipy.stats import randint, uniform

In [7]:
def random_search(f, vardists, num_iterations):
    best_solution = None
    best_cost = float("inf")
    history = []
    t = trange(num_iterations, desc="Best cost", leave=True)
    for _ in t:
        solution = []
        for vdist in vardists:
            if isinstance(vdist, (list, tuple)):
                value = random.choice(vdist)
            else:
                value = vdist.rvs(random_state=0)
            solution.append(value)

        cost = f(solution)
        history.append((solution, cost))
        if cost < best_cost:
            best_solution = solution
            best_cost = cost
        t.set_description(f"Best cost: {best_cost:.2e}")
        t.refresh()
    return best_solution, best_cost, history

In [8]:
vardists = (
    uniform(loc=0.1, scale=9.9),  # theta:  [0.1, 10]
    [1, 2, 3],  # mapping
    uniform(loc=-1, scale=1),  # alpha:  [-1, 0]
    uniform(loc=-1, scale=4),  # beta:   [-1, 3]
    uniform(loc=0.1, scale=1.9),  # lambda: [0.1, 2]
)

num_iterations = 24  # same as grid search

best_solution, best_fitness, history = random_search(
    run_trial, vardists, num_iterations
)

Best cost: 9.97e-03: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:19<00:00,  1.25it/s]


# Bayesian Optimization

In [9]:
from skopt import gp_minimize
from skopt.space import Integer, Real

In [10]:
varbound = [[0.1, 10], [1, 3], [-1, 0], [-1, 3], [0.1, 2]]
vartype = ("real", "int", "real", "real", "real")

# Convert variable types to skopt Space objects
dimensions = []
for i, typ in enumerate(vartype):
    if typ == "real":
        dimensions.append(Real(varbound[i][0], varbound[i][1]))
    elif typ == "int":
        dimensions.append(Integer(varbound[i][0], varbound[i][1]))

result = gp_minimize(
    run_trial,
    dimensions=dimensions,
    n_calls=24,
    random_state=1,
    verbose=False,
    n_jobs=1,
)

print(f"Best cost: {result.fun: .2e}")

Best cost:  1.23e-02


# Genetic Algorithm

In [11]:
from geneticalgorithm2 import geneticalgorithm2 as ga

In [12]:
algorithm_param = {
    "max_num_iteration": 3,
    "population_size": 10,
    "mutation_probability": 0.5,
    "selection_type": "tournament",
}

varbound = [[0.1, 10], [1, 3], [-1, 0], [-1, 0], [0.1, 2]]
vartype = ("real", "int", "real", "real", "real")
model = ga(
    function=run_trial,
    dimension=len(vartype),
    variable_type=vartype,
    variable_boundaries=varbound,
    algorithm_parameters=algorithm_param,
)
result = model.run(progress_bar_stream="stdout", no_plot=True)

print(f"Best cost: {model.report[-1]: .2e}")


Set: Average time of function evaluating (secs): 0.84453866481781 (total = 8.4453866481781)

Best score before optimization: 0.00602125167829
                                                                                                                                                                                                        
 The best found solution:
 [ 3.08327631  3.         -0.32913013 -0.65954122  1.35035744]

 Objective function:
 0.00434509117328

 Used generations: 3
 Used time: 16.8 seconds
Best cost:  4.35e-03
